# EVM Symbolic Execution

Our goal is to add symbolic execution to a rust based implementation of the EVM.

## Symbolic execution in the large
Symbolic execution is a general (not EVM specific) program analysis technique.
- [Symbolic Execution Introduction](https://en.wikipedia.org/wiki/Symbolic_execution) read the intro paragraph and the example.
- [Symbolic Execution Lecture](https://www.youtube.com/watch?v=yRVZPvHYHzw) watch the whole thing. You should follow most all of it before you continue.

Symbolic execution also has applications to the EVM. It can be used to find conditions that lead to assertion violations as well as other use cases. 
- [HEVM Symbolic Execution](https://fv.ethereum.org/2020/07/28/symbolic-hevm-release/) read the full post but do not attempt to understand everything.
- [Dapptools testing integration](https://youtu.be/N9pJ9JieX10?t=990) Watch the linked timestamp to see how dapptools uses hevm to symbolically search for assertion failures

All symbolic execution implementations rely on an SMT Solver. You should have a basic understanding of the SMT Solver's role from the Symbolic Execution Lecture. We will be using Z3 for our examples because it has a nice python library. For our actual implementation we will be as solver agnostic as possible.
- [Z3 tutorial](https://theory.stanford.edu/~nikolaj/programmingz3.html) Read all of section 2 and sections 3.1-3.4. We will not be too deep in the weeds of the solver so focus on intuition instead of formal understanding. Feel free to play with some of the code samples.

## Vocabulary
- Concrete value - A value that is known at runtime. When concrete values are combined with other concrete values, the result is also concrete.
- Symbolic value - A value that is not known at runtime. Symbolic values still have known and fixed types. When symbolic values are combined with both concrete and symbolic values, the result is also symbolic.
- Memory model - How memory is represented so that it can hold symbolic values. "Memory" in the general sense of all readable and writable storage locations. I.e. for the EVM, the memory model refers to memory, storage, calldata, and code.
- Concrete instruction - An instruction that only operates on concrete values.
- Symbolic Instruction - An instruction that can operates on symbolic values, concrete values, and combinations of the two.
- (Path) Constraint set - The set of constraints collected while executing the program. They constrain the set of possible concrete values that the symbolic values can represent.
- Machine state - Abstract term referring to the current state of (a single) execution of the program. This includes the state of the memory mode, the constraint set, program counter, etc...
- Interpreter - The function that takes in one machine state, executes an instruction, and returns multiple machine states. If the returning multiple machine states part confuses you, that's ok, assume it returns a single updated machine state for now.
- Runtime - Abstract term referring to the code orchestrating the memory model, interpreter, and machine state to compute.
- Solver - The SMT solver that is used to check if constraints can be satisfied.
- Satisfied - There exists a possible solution to the set of constraints given. A solution can be used to replace symbolic values with concrete values that would lead to the current machine state.
- Unsatisfied - The solver could not find a solution to the set of constraints given. An unsatisfied set of constraints is used to assume that the current machine state cannot exist and it should not be explored further. Note that this doesn't mean that one doesn't exist and can be a false negative.


## Symbolic execution in the small
We are going to walk through some examples of implementing symbolic execution against an EVM-like assembly language. 

All symbolic execution implementations follow a common pattern of taking an existing program runtime that executes over concrete values and modifying it such that...
- All concrete readable and writable values are replaced with a memory model that handles symbolic values.
- The interpreter executes symbolic instructions
- The machine state tracks path constraints
- The runtime can pass a set of path constraints to a solver to check for satisfiability
- The Runtime can handle multiple concurrent machine states (ok if you don't understand why right now)

We are going to work our way up from simpler machines to more complex machines adding incremental symbolic execution functionality as needed.

One important thing to note is that there is not one way to implement symbolic execution and symbolic execution is not all or nothing. You can force parts of the runtime to only execute over concrete values and in fact frequently we do for performance reasons.

Note that all python code runs on Python 3.10 and we use the library [z3-solver](https://pypi.org/project/z3-solver/) to interact with Z3.

### Arithmetic operations on stack values
Our first example will be assuming that our language has ADD and SUB instructions that operate on the stack. For simplicity sake, we will assume that we can store actual integers on the stack and not fixed width bit representations of integers. We consider the "result" of the program to be the item on the top of the stack when the program ends. Take this simple implementation.

In [1]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int

class StackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can compute things!

In [2]:
# (3 + 2) - 1 -> 4
m = StackArith([PUSH(1), PUSH(2), PUSH(3), ADD(), SUB()])
m.run()

4

Now what happens when we put symbolic values on the stack. Note that we don't have to change the implementation. This is because the z3 objects implement their own `+` method.

In [3]:
from z3 import *
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB()])
m.run()

z + y - x

The result of the computation is also symbolic. Note that the z3 library prints a nice string representation of the compound symbolic value. The actual type of the python value is from the Z3 package and the sort is still int. "Sorts"  are the Z3 equivalent of type (is this actually true?). Really "sort" comes from smtlib and not Z3 but that's getting ahead of ourselves.

In [4]:
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB()])
res = m.run()
print(res.__class__)
print(res.sort())

<class 'z3.z3.ArithRef'>
Int


We can also include concrete values on the stack, so the returned value contains both symbolic and concrete values.

In [5]:
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(3), ADD(), SUB()])
m.run()

3 + y - x

Just computing on symbolic values can be helpful in certain circumstances, but we want to be able to ask the question "what inputs do I have to pass this program for it to give this output". Let's add an additional instruction `ASSERT(n)` that adds the constraint that the top item on the stack is equal to its argument.

In [6]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class ASSERT:
    n: int

class SymStackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX We add the set of constraints to the machine state
        # Solver is the object from the z3 package that can be used
        # to hold constraints
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case ASSERT(n):
                # XXX We add the constraint to the machine state.
                self.constraints.add(self.stack[-1] == n)
            case _:
                assert False
            
    # TODO change to (the concrete value, the symbolic value, the model) here and for other examples
    # Returns: (The symbolic value, the concrete value (if satisfiable), the model (if satisfiable))
    def run(self):
        while self.pc < len(self.code):
            self.step()
            
        # XXX We check if the set of constraints is satisfiable.
        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)
            return rv, eval_rv, model
        else:
            # The set of constraints is not satisfiable. There is no
            # model or concrete value to return
            return rv, None, None

In [7]:
m = SymStackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB(), ASSERT(5)])
m.run()

(z + y - x, 5, [y = 0, x = 0, z = 5])

The answer here is not terribly interesting, but it tells us that replacing x, y, and z with 0, 0, and 5 respectively will result in the final value on the stack being equal to 5. We can confirm this by re-running the same machine with the concrete values.

In [8]:
m = SymStackArith([PUSH(0), PUSH(0), PUSH(5), ADD(), SUB()])
m.run()

(5, 5, [])

What happens when the solution is not satisfiable? Let's try a basic example where we assert one symbolic value to be equal to two different concrete values

In [9]:
m = SymStackArith([PUSH(Int('x')), ASSERT(1), ASSERT(2)])
m.run()

(x, None, None)

The constraint solver tells us that no possible concret value of `x` can satisfy both constraints.

### Read only memory

Our next example will be adding readonly memory to the machine. In the concrete machine, memory will be represented with a python list that is passed into the machine. The machine will not modify the list. We will add an `MLOAD` instruction to read from the memory.

In [10]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

class ReadonlyMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory[self.stack.pop()])
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

In this modified example, we now put the initial operand in memory and load it from memory before performing the addition.

In [11]:
mem = [0 for x in range(5)]
mem[4] = 3
# (mem[4] + 2) - 1
# -> (3 + 2) - 1 
# -> 4
m = ReadonlyMem([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB()], mem)
m.run()

4

Converting stack based arithmetic operations to symbolic was rather straight forward. They are operations, they take operands. In order to pass symbolic operands instead of concrete operands, the stack has to be able to hold symbolic values. 

Symbolic memory is more complicated because the memory can hold symbolic values, the memory can be indexed at symbolic values, and even the length of memory can be symbolic. This can be hard to conceptualize so we will incrementally add symbolic features to concrete memory. This is a common theme when figuring out how to implement symbolic execution. We start with adding simple symbolic features and incrementally add more symbolic features.

The simplest symbolic feature we can add is letting memory store symbolic values. Because the only major change from `ReadonlyMem` is that the stack and memory can hold symbolic values, we add an assertion that the memory index is concrete in `MLOAD`.

In [12]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class ASSERT:
    n: int

class SymReadonlyMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                idx = self.stack.pop()
                assert isinstance(idx, int), 'MLOAD: requires concrete index'
                self.stack.append(self.memory[idx])
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

Let's modify the previous example to now read a symbolic value from memory.

In [13]:
mem = [0 for x in range(5)]
mem[4] = Int('z')
m = SymReadonlyMem([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB(), ASSERT(4)], mem)
m.run()

(z + 2 - 1, 4, [z = 3])

We now get runtime errors when we try to read memory at a symbolic index

In [14]:
mem = [0 for x in range(5)]
m = SymReadonlyMem([PUSH(Int('x')), MLOAD()], mem)
try:
    m.run()
except AssertionError as e:
    print(e)

MLOAD: requires concrete index


If we want to read from memory at symbolic indices, we can't use a python list to represent memory. We have to use an SMT solver primitive called an uninterpreted function. Re-read [this section](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-euf--equality-and-uninterpreted-functions) if you need a refresher. The tl;dr is that uninterpreted functions can be used to model constraints on [mathematical functions](https://en.wikipedia.org/wiki/Function_(mathematics)) without worrying about the internals of how the function might be implemented. (TODO this could probably use some work) 

Because the uninterpreted function has to be defined over its entire domain, and the domain is the entire set of integers, we are saying that this machine has unbounded memory. We'll solve that later.

In [15]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class ASSERT:
    n: int

class SymReadonlyMemSymIndex:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX
        self.memory = Function('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory(self.stack.pop()))
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

How to read output from the constraint solver for an uninterpreted function. For the following example, `f=[1->2, else -> 0]` can be read as `f(1)` returns `2` and for all other values in the input set, `f` returns 0. Note that this implicitly covers the first added constraint `f(0) == 0`. If you don't explicitly see a constraint you added in the output, check if the "else" covers it.

In [16]:
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(0) == 0)
s.add(f(1) == 2)
s.check()
s.model()

[f = [1 -> 2, else -> 0]]

We can execute the same example as before where we read the first operand from memory index 4, but this time we don't have to explicitly set that index in memory to being a symbolic value.

Note that the model says `memory = [else -> 3]`. Assume that memory is of size 5, this means memory would look like `[3, 3, 3, 3, 3]`. In the concrete machine, we set memory to  `[0, 0, 0, 0, 3]`. Both are valid initial machine states for the code we executed. The only thing that matters is that `memory[4] == 3`. The constraint solver is not guaranteed to give you back a particular valid machine state, just _a_ valid machine state. (TODO is "machine state" the best word to use in this context)

In [17]:
m = SymReadonlyMemSymIndex([
    PUSH(1), 
    PUSH(2), 
    PUSH(4), MLOAD(), 
    ADD(), 
    SUB(), 
    ASSERT(4)
])
m.run()

(memory(4) + 2 - 1, 4, [memory = [else -> 3]])

Let's now read from a symbolic memory index. The model resolves to the same initial memory state but says we read from index 0. Again, this is a _different_ initial machine state, but it is a _valid_ initial machine state. We just have to read the value 3 from a memory index.

In [18]:
m = SymReadonlyMemSymIndex([
    PUSH(1),
    PUSH(2),
    # XXX
    PUSH(Int('idx')), MLOAD(),
    ADD(),
    SUB(),
    ASSERT(4)
])
m.run()

(memory(idx) + 2 - 1, 4, [idx = 0, memory = [else -> 3]])

### Read/Write memory

Our next example takes the read only memory and adds writing. We will add an `MSTORE` instruction to write to memory.

Modifying the concrete machine to write to memory is straight forward.

In [19]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

class RWMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                # XXX
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory[idx] = val
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can now write the first operand to memory and then read it back to the stack before operating on it.

In [20]:
mem = [0 for x in range(5)]

m = RWMem([
    PUSH(1),
    PUSH(2),
    # XXX Write operand to memory
    PUSH(3), PUSH(0), MSTORE(),
    # XXX Read operand from memory
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
], mem)

m.run()

4

To make memory writable, we need to use a different SMT solver primitive -- [arrays](https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-arrays). Just like uninterpreted functions, arrays can be used to model mathematical functions, but arrays allow the function to be "updatable". The "update" does not modify the original function. It creates a new and updated function that has one input output pair changed. In our machine, on every write instruction, we replace the memory array with the new updated memory array. (TODO -- "I think update sounds like an in place change. I prefer to talk about it like an infinite map with the pure function store that takes an array and returns the modified array")

In [21]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ASSERT:
    n: int

class SymRWMem:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX
        self.memory = Array('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                # XXX
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory = Store(self.memory, idx, val)
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

We can now write a symbolic value to a concrete index for the first operand.

In [22]:
m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('x')), PUSH(0), MSTORE(),
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
    ASSERT(4)
])
m.run()

(Store(memory, 0, x)[0] + 2 - 1, 4, [x = 3])

We can even write a symbolic value to a symbolic index for the first operand. Note that the model solves for the symbolic value but does not have a value for the index. The model does not need to determine a concrete value for the index because for the model to be satisfiable, the operand just has to written to and read from the same index. The more formal term for the model "not having a value" is "does not have an interpretation".

In [23]:
idx = Int('idx')

m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('val')), PUSH(idx), MSTORE(),
    PUSH(idx), MLOAD(),
    ADD(), 
    SUB(),
    ASSERT(4)
])

m.run()

(Store(memory, idx, val)[idx] + 2 - 1, 4, [val = 3])

If we need a concrete value for the index, we can ask the model gives us a "default interpretation" for the expression by setting the `model_completion` flag to true.

In [24]:
idx = Int('idx')

m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('val')), PUSH(idx), MSTORE(),
    PUSH(idx), MLOAD(),
    ADD(), 
    SUB(),
    ASSERT(4)
])

(_, _, model) = m.run()

model.eval(idx, model_completion=True)

0

### Conditionals

Our next machine adds conditionals through the `ISZERO` instruction. `ISZERO`'s concrete implementation is straight forward.

In [25]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

class Cond:
    def __init__(self, code, memory=[]):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory[idx] = val
            case ISZERO():
                # XXX
                val = self.stack.pop()
                self.stack.append(1 if val == 0 else 0)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

Let's modify our original example to check the expected answer in the machine.

In [26]:
# ((3 + 2) - 1)) - 4 == 0
m = Cond([
    PUSH(1), 
    PUSH(2), 
    PUSH(3), 
    ADD(), 
    SUB(),
    # XXX We expect our solution to be 4, try to zero out the solution
    PUSH(4),
    SUB(),
    ISZERO()
])
m.run()

1

There is an "if-then-else" SMT solver primitive. In Z3, this is the `If` function. Note, the Z3 `If` function represents the smtlib `ite` function (we'll get to smtlib later). The `If` function can be used to build a compound symbolic value. The compound symbolic value can be thought of as implicitly carrying a constraint.

In [27]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

@dataclass
class ASSERT:
    n: int

class SymCond:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = Array('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory = Store(self.memory, idx, val)
            case ISZERO():
                # XXX
                val = self.stack.pop()
                self.stack.append(If(val == 0, 1, 0))
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
            return rv, eval_rv, model
        else:
            return rv, None, None

We can now substitute a symbolic value into the operand and later assert the result of the `ISZERO` check. This later assertion can be thought of as setting that the implicit constraint in the `If` expression does hold.

In [28]:
m = SymCond([
    PUSH(1), 
    PUSH(2), 
    PUSH(Int('x')), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    # XXX
    ASSERT(1)
])
m.run()

(If(4 - (x + 2 - 1) == 0, 1, 0), 1, [x = 3])

We can instead assert that the constraint does not hold. Remember there are many valid concrete values `x` could resolve to in this case, but the constraint solver will just pick one for us.

In [29]:
m = SymCond([
    PUSH(1), 
    PUSH(2), 
    PUSH(Int('x')), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    # XXX
    ASSERT(0)
])
m.run()

(If(4 - (x + 2 - 1) == 0, 1, 0), 0, [x = 4])

### Conditional Jumps

Our next example adds conditional jumps via the `JUMPI` instruction. Note `JUMPI` takes an absolute position to set the program counter to.

We'll also add a `STOP` opcode that halts execution to make the example easier to write

In [30]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

@dataclass
class JUMPI:
    pass

@dataclass
class STOP:
    pass

class CondJump:
    def __init__(self, code, memory=[]):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        
        # XXX
        if instruction == JUMPI():
            dest = self.stack.pop()
            cond = self.stack.pop()
            self.pc = self.pc + 1 if cond == 0 else dest
            return
        
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory[idx] = val
            case ISZERO():
                val = self.stack.pop()
                self.stack.append(1 if val == 0 else 0)
            case STOP():
                # XXX
                self.pc = len(self.code)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can now modify our previous example to take a jump if the `ISZERO` check returns true.

In [31]:
m = CondJump([
    PUSH(1), 
    PUSH(2), 
    PUSH(3), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    # XXX 
    PUSH(12),
    JUMPI(),
    # The condition did not hold
    PUSH(100),
    STOP(),
    # pc = 12
    # The condition did hold
    PUSH(200)
])
m.run()

200

Symbolic machine conditional jumps require a larger deviation from the concrete machine. When the conditional argument to the jump is symbolic, our machine must take both paths. This means that our machine must now keep track of multiple machine states and choose which machine state to execute. We do not allow symbolic arguments to the jump destination. If we did allow symbolic arguments to the jump destination, we would have to split the machine state into `len(self.code)` copies at every symbolic jump. This is a common pattern in symbolic execution where we sometimes choose to just not make the full implementation symbolic for performance reasons.

In [32]:
from dataclasses import dataclass
from z3 import *
import copy 

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

@dataclass
class JUMPI:
    pass

@dataclass
class STOP:
    pass

@dataclass
class ASSERT:
    n: int
        
# TODO add type definitions
@dataclass
class MachineState:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineState(
            pc=self.pc,
            stack=self.stack.copy(),
            # Memory is represented as an SMT Array which is immutable
            # and does not need to be copied
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )
        
class SymCondJump:
    def __init__(self, code):
        # XXX all machine states can share the same code because
        # code is read only
        self.code = code
        
        # XXX we now store multiple machine states
        self.machines = [
            MachineState(
                pc=0, 
                stack=[], 
                memory=Array('memory', IntSort(), IntSort()), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]
        
        # XXX JUMPI now creates two different machine states        
        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            
            other_machine = copy.copy(machine)
            
            # other machine does take the jump
            other_machine.pc = dest
            other_machine.constraints.append(cond != 0)
            
            # machine does not take the jump
            machine.pc = machine.pc + 1
            machine.constraints.append(cond == 0)
            
            self.machines.append(other_machine)

            return        
        
        machine.pc += 1
        
        match instruction:
            case ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
            case SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
            case PUSH(n):
                machine.stack.append(n)
            case MLOAD():
                machine.stack.append(machine.memory[machine.stack.pop()])
            case MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
            case ISZERO():
                val = machine.stack.pop()
                machine.stack.append(If(val == 0, 1, 0))
            case STOP():
                machine.pc = len(self.code)                  
            case ASSERT(n):
                machine.constraints.add(machine.stack[-1] == n)              
            case _:
                assert False        
    
    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        
    
    def run(self):
        # XXX We must choose a machine state to pass to step
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
        
        rvs = []
        
        for m in self.machines:
            rv = m.stack[-1]
            if m.constraints.check() == sat:
                model = m.constraints.model()
                eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
                rvs.append((rv, eval_rv, model))
            else:
                rvs.append((rv, None, None))
            
        return rvs     

When we run the same example from the concrete case, we see that we do indeed get two machine states output, one where the jump is not taken (final value 100) and one where the jump is taken (final value 200). However, the output from the not taking the jump case indicates that while the final concrete value on the stack was 100, the constraint set was not satisfiable. This means that it is not actually possible to reach this machine state. We can actually check if the model can be satisfied when splitting the machine states and not follow unsatisfiable machine states.

In [33]:
m = SymCondJump([
    PUSH(1), 
    PUSH(2), 
    PUSH(3), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    PUSH(12),
    JUMPI(),
    PUSH(100),
    STOP(),
    PUSH(200)
])
m.run()

[(100, None, None), (200, 200, [])]

In [34]:
from dataclasses import dataclass
from z3 import *
import copy 

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ISZERO:
    pass

@dataclass
class JUMPI:
    pass

@dataclass
class STOP:
    pass

@dataclass
class ASSERT:
    n: int
        
# TODO add type definitions
@dataclass
class MachineState:
    pc: int
    stack: any
    memory: any
    constraints: any
        
    def __copy__(self):
        return MachineState(
            pc=self.pc,
            stack=self.stack.copy(),
            # Memory is represented as an SMT Array which is immutable
            # and does not need to be copied
            memory=self.memory,
            constraints=copy.copy(self.constraints)
        )
        
class SymCondJump:
    def __init__(self, code):
        self.code = code
        
        self.machines = [
            MachineState(
                pc=0, 
                stack=[], 
                memory=Array('memory', IntSort(), IntSort()), 
                constraints=Solver()
            )
        ]
        
    def step(self, machine):
        instruction = self.code[machine.pc]
        
        if instruction == JUMPI():
            dest = machine.stack.pop()
            assert isinstance(dest, int), 'JUMPI: requires concrete destination'
            cond = machine.stack.pop()
            
            other_machine = copy.copy(machine)
            
            # other machine does take the jump
            other_machine.pc = dest
            other_machine.constraints.append(cond != 0)
            
            # machine does not take the jump
            machine.pc = machine.pc + 1
            machine.constraints.append(cond == 0)
            
            # XXX remove machine from the list of machines if it is not satisfiable
            if machine.constraints.check() == unsat:
                self.machines.remove(machine)
            
            # XXX only add other_machine to the list of machines if it is satisfiable
            if other_machine.constraints.check() == sat:
                self.machines.append(other_machine)

            return        
        
        machine.pc += 1
        
        match instruction:
            case ADD():
                machine.stack.append(machine.stack.pop() + machine.stack.pop())
            case SUB():
                machine.stack.append(machine.stack.pop() - machine.stack.pop())
            case PUSH(n):
                machine.stack.append(n)
            case MLOAD():
                machine.stack.append(machine.memory[machine.stack.pop()])
            case MSTORE():
                idx = machine.stack.pop()
                val = machine.stack.pop()
                machine.memory = Store(machine.memory, idx, val)
            case ISZERO():
                val = machine.stack.pop()
                machine.stack.append(If(val == 0, 1, 0))
            case STOP():
                machine.pc = len(self.code)                  
            case ASSERT(n):
                machine.constraints.add(machine.stack[-1] == n)              
            case _:
                assert False        
    
    def choose_machine(self):
        machine = None
        
        for m in self.machines:
            if m.pc < len(self.code):
                machine = m
                break
                
        return machine        
    
    def run(self):
        machine = self.choose_machine()
        
        while machine:
            self.step(machine)
            machine = self.choose_machine()            
        
        rvs = []
        
        for m in self.machines:
            rv = m.stack[-1]
            if m.constraints.check() == sat:
                model = m.constraints.model()
                eval_rv = rv if isinstance(rv, int) else model.eval(rv)            
                rvs.append((rv, eval_rv, model))
            else:
                rvs.append((rv, None, None))
            
        return rvs     

Now the example with only concrete values returns one final machine state.

In [35]:
m = SymCondJump([
    PUSH(1), 
    PUSH(2), 
    PUSH(3), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    PUSH(12),
    JUMPI(),
    PUSH(100),
    STOP(),
    PUSH(200)
])
m.run()

[(200, 200, [])]

Now let's see what happens when our branching condition is symbolic. When we make the value that we use to check the answer symbolic, we get two possible answers that lead to the separate respective paths. 

In [36]:
m = SymCondJump([
    PUSH(1), 
    PUSH(2), 
    PUSH(3), 
    ADD(), 
    SUB(),
    PUSH(Int('ans')),
    SUB(),
    ISZERO(),
    PUSH(12),
    JUMPI(),
    PUSH(100),
    STOP(),
    PUSH(200)
])
m.run()

[(100, 100, [ans = 5]), (200, 200, [ans = 4])]

Intead of making the answer check symbolic, we can make an operand symbolic

In [37]:
m = SymCondJump([
    PUSH(1), 
    PUSH(2), 
    PUSH(Int('x')), 
    ADD(), 
    SUB(),
    PUSH(4),
    SUB(),
    ISZERO(),
    PUSH(12),
    JUMPI(),
    PUSH(100),
    STOP(),
    PUSH(200)
])
m.run()

[(100, 100, [x = 4]), (200, 200, [x = 3])]

We can make both symbolic

In [38]:
m = SymCondJump([
    PUSH(1), 
    PUSH(2), 
    PUSH(Int('x')), 
    ADD(), 
    SUB(),
    PUSH(Int('ans')),
    SUB(),
    ISZERO(),
    PUSH(12),
    JUMPI(),
    PUSH(100),
    STOP(),
    PUSH(200)
])
m.run()

[(100, 100, [ans = 2, x = 0]), (200, 200, [x = 0, ans = 1])]

We can make all operands and the answer symbolic. Hopefully you can start to see how the different pieces are starting to come together.

In [39]:
m = SymCondJump([
    PUSH(Int('x')), 
    PUSH(Int('y')), 
    PUSH(Int('z')), 
    ADD(), 
    SUB(),
    PUSH(Int('ans')),
    SUB(),
    ISZERO(),
    PUSH(12),
    JUMPI(),
    PUSH(100),
    STOP(),
    PUSH(200)
])
m.run()

[(100, 100, [ans = 1, z = 0, y = 0, x = 0]),
 (200, 200, [z = 0, y = 0, x = 0, ans = 0])]

### A more realistic model of memory using Bit Vectors
 
We've been assuming we can store arbitrary precision integers on our machine's stack. Some [virtual machines](https://docs.oracle.com/javase/specs/jvms/se7/html/jvms-2.html#jvms-2.6.2) do let you store a rich set of primitive types on the stack. However, the EVM stores fixed-width bit vectors. A 256 bit (32 byte) word size "was chosen to facilitate the Keccak256 hash scheme and elliptic-curve computations (TODO cite yellow paper -- section 9.1)." The EVM is big endian.

Using bit vectors instead of integers makes it easier to model EVM semantics as the EVM is defined in terms of operations on bit vectors. I.e. in the EVM, `ADD` is unsigned addition modulo the word size of the machine, not addition over arbitrary precision integers. We could add additional constraints to the result of the integer addition to make the result equivalent to EVM semantics, but for now we want to directly model the lower level operation.

More importantly, we can't faithfully model memory addressability using integers. 

The EVM has 4 different memory locations: the stack, calldata, memory, and storage. Each uses 32 byte words but has different access semantics.

Each stack item is a full word but we can push variable byte amounts that are then padded to a full word. The following python snippet mimics the EVM set of `PUSH<N>` instructions.

In [40]:
stack = []

def push(val):
    padded = bytes.fromhex(val).rjust(32, b'\x00').hex()
    stack.append(padded)

push('0a')
push('01')
push('deadbeef')
push('a1deadbeef')

print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())

000000000000000000000000000000000000000000000000000000a1deadbeef
00000000000000000000000000000000000000000000000000000000deadbeef
0000000000000000000000000000000000000000000000000000000000000001
000000000000000000000000000000000000000000000000000000000000000a


Memory and calldata are byte addressable arrays and we read full words from them. This means that reads and writes at adjacent indices touch overlapping regions of memory. Calldata is not writable but memory is written to in full word increments or as a single byte. `MSTORE8` writes the low byte of the operand. The following python snippet mimics the EVM instructions that write and read from memory. Calldata can't be written to and reading from calldata is basically the same as reading from memory.

In [41]:
mem = []

def grow_to(length):
    while len(mem) < length:
        mem.append(0x00)

def mload(idx):
    grow_to(idx + 32)
    return bytes(mem[idx: idx + 32]).hex()

def mstore(idx, val):
    grow_to(idx + 32)
    val = bytes.fromhex(val)    
    for offset in range(32):
        mem[idx + offset] = val[offset]

def mstore8(idx, val):    
    grow_to(idx + 1)
    low_byte = bytes.fromhex(val)[31]
    mem[idx] = low_byte
    

mstore(0, '00' * 32)
print(bytes(mem).hex())
print(mload(0))
print()

mstore(1, '11' * 32)
print(bytes(mem).hex())
print(mload(0))
print(mload(1))
print()

mstore(32, '22' * 32)
print(bytes(mem).hex())
print(mload(0))
print(mload(1))
print(mload(32))
print()

mstore8(0, '33' * 32)
mstore8(1, '44' * 32)
mstore8(2, '55' * 32)
print(bytes(mem).hex())
print(mload(0))

0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000

001111111111111111111111111111111111111111111111111111111111111111
0011111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111

00111111111111111111111111111111111111111111111111111111111111112222222222222222222222222222222222222222222222222222222222222222
0011111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111122
2222222222222222222222222222222222222222222222222222222222222222

33445511111111111111111111111111111111111111111111111111111111112222222222222222222222222222222222222222222222222222222222222222
3344551111111111111111111111111111111111111111111111111111111111


Storage is a key value store that maps words to words. Storage reads are _not_ byte addressable. This means that reads and writes at adjacent storage locations do not touch overlapping regions of memory. The following snippet mimics reading and writing from EVM storage.

In [42]:
storage = {}

def sload(idx):
    if idx not in storage:
        return '00' * 32
    else:
        return storage[idx]

def sstore(idx, val):
    storage[idx] = val
    
print(sload('00' * 32))
print()

sstore('00' * 32, '11' * 32)
print(sload('00' * 32))
print()

sstore(('00' * 31) + '01', '22' * 32)
print(sload('00' * 32))
print(sload(('00' * 31) + '01'))
print()

0000000000000000000000000000000000000000000000000000000000000000

1111111111111111111111111111111111111111111111111111111111111111

1111111111111111111111111111111111111111111111111111111111111111
2222222222222222222222222222222222222222222222222222222222222222



For our example machine, we will be using 4 byte word size that is byte addressable.

### Bit vector arithmetic operations on stack values

Python supports most of its bitwise operations directly on integer values so we will still store and operate on integers. For addition and subtraction where python does not support the direct bitwise operation, we can bitwise and the result with `MAX_INT` to keep the result in range. This is how [py-evm](https://github.com/ethereum/py-evm/blob/026ee20f8d9b70d7c1b6a4fb9484d5489d425e54/eth/vm/logic/arithmetic.py) implements bitwise addition and subtraction.

In [43]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
# XXX
MAX_INT = 2**32 - 1

class BitVecStackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                # XXX
                self.stack.append(self.stack.pop() + self.stack.pop() & MAX_INT)
            case SUB():
                # XXX
                self.stack.append(self.stack.pop() - self.stack.pop() & MAX_INT)
            case PUSH(n):
                self.stack.append(n)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can still compute values with the machine, but now the results are restricted to as if they are computed over fixed width integers.

In [44]:
print(BitVecStackArith([
    PUSH(MAX_INT),
    PUSH(1),
    ADD()
]).run())

print(BitVecStackArith([
    PUSH(1),
    PUSH(0),
    SUB()
]).run())

0
4294967295


The symbolic version of this machine will still need the bitwise and check when operating on two integers, but when at least one of the operands is symbolic, the constraint solver will ensure that the compound symbolic expression is in the right range. The only other change we have to make is when specifying bit vector width when declaring symbolic variables.

In [45]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class ASSERT:
    n: int
        
# XXX
MAX_INT = 2**32 - 1

class SymBitVecStackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                # XXX
                a = self.stack.pop()
                b = self.stack.pop()
                if isinstance(a, int) and isinstance(b, int):
                    self.stack.append(a + b & MAX_INT)
                else:
                    self.stack.append(a + b)
            case SUB():
                # XXX
                a = self.stack.pop()
                b = self.stack.pop()
                if isinstance(a, int) and isinstance(b, int):
                    self.stack.append(a - b & MAX_INT)
                else:
                    self.stack.append(a - b)
            case PUSH(n):
                self.stack.append(n)
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)
            case _:
                assert False
            
    def run(self):
        while self.pc < len(self.code):
            self.step()
            
        rv = self.stack[-1]
        if self.constraints.check() == sat:
            model = self.constraints.model()
            eval_rv = rv if isinstance(rv, int) else model.eval(rv)
            return rv, eval_rv, model
        else:
            return rv, None, None

Our original example still works.

In [46]:
m = SymBitVecStackArith([
    PUSH(1),
    PUSH(2),
    # XXX
    PUSH(BitVec('x', 32)),
    ADD(),
    SUB(),
    ASSERT(4)
])
m.run()

(x + 2 - 1, 4, [x = 3])

The new machine models overflow correctly

In [47]:
m = SymBitVecStackArith([
    PUSH(1),
    PUSH(BitVec('x', 32)),
    ADD(),
    ASSERT(0)
])
m.run()

(x + 1, 0, [x = 4294967295])

### Bit vector memory concrete indexing

In [48]:
from z3 import *

memory = []

def grow_to(length):
    while len(memory) < length:
        memory.append(0x00)
        
def coerce_byte(byte):
    if isinstance(byte, int):
        return BitVecVal(byte, 8)
    
    return byte

def mload(idx):
    assert isinstance(idx, int), 'MLOAD: requires concrete index'
    
    grow_to(idx + 4)
    
    rv = coerce_byte(memory[idx])
    for offset in range(1, 4):
        # XXX
        rv = Concat(rv, coerce_byte(memory[idx + offset]))
    return rv

def mstore(idx, val):
    assert isinstance(idx, int), 'MSTORE: requires concrete index'
    
    grow_to(idx + 4)
    
    if isinstance(val, str):
        val = bytes.fromhex(val)
        for offset in range(4):
            memory[idx + offset] = val[offset]
    else:
        for offset in range(4):
            # XXX
            start_idx = 8 * offset
            memory[idx + offset] = Extract(7 + start_idx, start_idx, val)

def mstore8(idx, val):    
    assert isinstance(idx, int), 'MSTORE8: requires concrete index'
    
    grow_to(idx + 1)
    
    if isinstance(val, str):
        low_byte = bytes.fromhex(val)[3]
    else:
        # XXX
        low_byte = Extract(7, 0, val)
        
    memory[idx] = low_byte

In [49]:
mstore(0, '00' * 4)
print(memory)
print(mload(0))
print()

mstore(1, '11' * 4)
print(memory)
print(mload(0))
print(mload(1))
print()

mstore(4, '22' * 4)
print(memory)
print(mload(0))
print(mload(1))
print(mload(4))
print()

mstore8(0, '33' * 4)
mstore8(1, '44' * 4)
mstore8(2, '55' * 4)
print(memory)
print(mload(0))

[0, 0, 0, 0]
Concat(Concat(Concat(0, 0), 0), 0)

[0, 17, 17, 17, 17]
Concat(Concat(Concat(0, 17), 17), 17)
Concat(Concat(Concat(17, 17), 17), 17)

[0, 17, 17, 17, 34, 34, 34, 34]
Concat(Concat(Concat(0, 17), 17), 17)
Concat(Concat(Concat(17, 17), 17), 34)
Concat(Concat(Concat(34, 34), 34), 34)

[51, 68, 85, 17, 34, 34, 34, 34]
Concat(Concat(Concat(51, 68), 85), 17)


### Bit vector memory read from symbolic addresses

In [50]:
from z3 import *

memory = Array('memory', BitVecSort(32), BitVecSort(8))
# XXX
mem_length = 0

def grow_to(length):
    global memory, mem_length
    while mem_length < length:
        # XXX
        memory = Store(memory, mem_length, 0)
        mem_length += 1
        
def coerce_byte(byte):
    if isinstance(byte, int):
        return BitVecVal(byte, 8)
    
    return byte

def mload(idx):
    # XXX
    # assert isinstance(idx, int), 'MLOAD: requires concrete index'
    
    # XXX
    # grow_to(idx + 4)
    
    # XXX
    rv = coerce_byte(If(idx >= mem_length, 0, memory[idx]))
    for offset in range(1, 4):
        # XXX
        v = coerce_byte(If(idx + offset >= mem_length, 0, memory[idx + offset]))
        rv = Concat(rv, v)
    return rv

def mstore(idx, val):
    global memory
    
    assert isinstance(idx, int), 'MSTORE: requires concrete index'
    
    grow_to(idx + 4)
    
    if isinstance(val, str):
        val = bytes.fromhex(val)
        for offset in range(4):
            # XXX
            memory = Store(memory, idx + offset, val[offset])
    else:
        for offset in range(4):
            start_idx = 8 * offset
            # XXX            
            memory = Store(memory, idx + offset, Extract(7 + start_idx, start_idx, val))

def mstore8(idx, val):
    global memory
    
    assert isinstance(idx, int), 'MSTORE8: requires concrete index'
    
    grow_to(idx + 1)
    
    if isinstance(val, str):
        low_byte = bytes.fromhex(val)[3]
    else:
        low_byte = Extract(7, 0, val)

    memory = Store(memory, idx, low_byte)

In [51]:
mstore(0, '00' * 4)
print(0x00000000)
print(simplify(mload(0)))
print()

mstore(1, '11' * 4)
print(0x00111111)
print(simplify(mload(0)))
print()
print(0x11111111)
print(simplify(mload(1)))
print()

mstore(4, '22' * 4)
print(0x00111111)
print(simplify(mload(0)))
print()
print(0x11111122)
print(simplify(mload(1)))
print()
print(0x22222222)
print(simplify(mload(4)))
print()

mstore8(0, '33' * 4)
mstore8(1, '44' * 4)
mstore8(2, '55' * 4)
print(0x33445511)
print(simplify(mload(0)))
print()

0
0

1118481
1118481

286331153
286331153

1118481
1118481

286331170
286331170

572662306
572662306

860116241
860116241



In [52]:
idx = BitVec('idx', 32)
val = mload(idx)
simplify(val)

Concat(If(8 <= idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(...,
                                        ...,
                                        ...),
                                        2,
                                        0),
                                        3,
                                        0),
                                        4,
                                        0),
                                        1,
                                        17),
                                        2,
                                        17),
                                        3,
                                        17),
                                        4,
                                        17),
                                        5,
                                        0),
                                        6,
                                        0),
                                        7,
                                        0),
                                        4,
                                        34),
                                        5,
                                        34),
                                        6,
                                        34),
                                  7,
                                  34),
                            0,
                            51),
                      1,
                      68),
                2,
                85)[idx]),
       If(8 <= 1 + idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(...,
                                        ...,
                                        ...),
                                        2,
                                        0),
                                        3,
                                        0),
                                        4,
                                        0),
                                        1,
                                        17),
                                        2,
                                        17),
                                        3,
                                        17),
                                        4,
                                        17),
                                        5,
                                        0),
                                        6,
                                        0),
                                        7,
                                        0),
                                        4,
                                        34),
                                        5,
                                        34),
                                        6,
                                        34),
                                  7,
                                  34),
                            0,
                            51),
                      1,
                      68),
                2,
                85)[1 + idx]),
       If(8 <= 2 + idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(...,
                                        ...,
                                        ...),
                                        2,
                                        0),
                                        3,
                                        0),
                                        4,
                                        0),
                                        1,
                                        17),
                                        2,
                                        17),
                                        3,
               

In [53]:
s = Solver()
s.add(idx == 0)
s.check()
print(0x33445511)
print(s.model().eval(val))

860116241
860116241


In [54]:
s = Solver()
s.add(idx == 4)
s.check()
print(0x22222222)
print(s.model().eval(val))

572662306
572662306


In [55]:
s = Solver()
s.add(idx == 5)
s.check()
print(0x22222200)
print(s.model().eval(val))

572662272
572662272


### Bit vector memory read/write symbolic addresses

In [56]:
from z3 import *

memory = Array('memory', BitVecSort(32), BitVecSort(8))
# XXX
mem_length = BitVecVal(0, 32)

def grow_to(length):
    global mem_length
    # XXX
    mem_length = If(length > mem_length, length, mem_length)
        
def coerce_byte(byte):
    if isinstance(byte, int):
        return BitVecVal(byte, 8)
    
    return byte

def mload(idx):
    rv = coerce_byte(If(idx >= mem_length, 0, memory[idx]))
    for offset in range(1, 4):
        v = coerce_byte(If(idx + offset >= mem_length, 0, memory[idx + offset]))
        rv = Concat(rv, v)
    return rv

def mstore(idx, val):
    global memory
    
    # XXX
    # assert isinstance(idx, int), 'MSTORE: requires concrete index'
    
    grow_to(idx + 4)
    
    if isinstance(val, str):
        val = bytes.fromhex(val)
        for offset in range(4):
            memory = Store(memory, idx + offset, val[offset])
    else:
        for offset in range(4):
            start_idx = 8 * offset            
            memory = Store(memory, idx + offset, Extract(7 + start_idx, start_idx, val))

def mstore8(idx, val):
    global memory
    
    # XXX
    # assert isinstance(idx, int), 'MSTORE8: requires concrete index'
    
    grow_to(idx + 1)
    
    if isinstance(val, str):
        low_byte = bytes.fromhex(val)[3]
    else:
        low_byte = Extract(7, 0, val)

    memory = Store(memory, idx, low_byte)

In [57]:
mstore(0, '00' * 4)
print(0x00000000)
print(simplify(mload(0)))
print()

mstore(1, '11' * 4)
print(0x00111111)
print(simplify(mload(0)))
print()
print(0x11111111)
print(simplify(mload(1)))
print()

mstore(4, '22' * 4)
print(0x00111111)
print(simplify(mload(0)))
print()
print(0x11111122)
print(simplify(mload(1)))
print()
print(0x22222222)
print(simplify(mload(4)))
print()

mstore8(0, '33' * 4)
mstore8(1, '44' * 4)
mstore8(2, '55' * 4)
print(0x33445511)
print(simplify(mload(0)))
print()

0
0

1118481
1118481

286331153
286331153

1118481
1118481

286331170
286331170

572662306
572662306

860116241
860116241



In [58]:
idx = BitVec('idx', 32)
val = mload(idx)
simplify(val)

Concat(If(8 <= idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(memory,
                                        0,
                                        0),
                                        1,
                                        0),
                                        2,
                                        0),
                                        3,
                                        0),
                                        1,
                                        17),
                                        2,
                                        17),
                                        3,
                                        17),
                                        4,
                                        34),
                                        5,
                                        34),
                                        6,
                                        34),
                                  7,
                                  34),
                            0,
                            51),
                      1,
                      68),
                2,
                85)[idx]),
       If(8 <= 1 + idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(memory,
                                        0,
                                        0),
                                        1,
                                        0),
                                        2,
                                        0),
                                        3,
                                        0),
                                        1,
                                        17),
                                        2,
                                        17),
                                        3,
                                        17),
                                        4,
                                        34),
                                        5,
                                        34),
                                        6,
                                        34),
                                  7,
                                  34),
                            0,
                            51),
                      1,
                      68),
                2,
                85)[1 + idx]),
       If(8 <= 2 + idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(memory,
                                        0,
                                        0),
                                        1,
                                        0),
                                        2,
                                        0),
                                        3,
                                        0),
                                        1,
                                        17),
                                        2,
                                        17),
                                        3,
                                        17),
                                        4,
                                        34),
                                        5,
                                        34),
                                        6,
                                        34),
                                  7,
                                  34),
                            0,
                            51),
                      1,
                      68),
                2,
                85)[2 + idx]),
       If(8 <= 3 + idx,
          0,
          Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(memory,
                                        0,
                                        0),
           

In [59]:
s = Solver()
s.add(idx == 0)
s.check()
print(0x33445511)
print(s.model().eval(val))

860116241
860116241


In [60]:
s = Solver()
s.add(idx == 4)
s.check()
print(0x22222222)
print(s.model().eval(val))

572662306
572662306


In [61]:
s = Solver()
s.add(idx == 5)
s.check()
print(0x22222200)
print(s.model().eval(val))

572662272
572662272


In [62]:
idx2 = BitVec('idx2', 32)
mstore(idx2, '55' * 4)
val = mload(idx2)
simplify(val)

Concat(If(If(4 + idx2 <= 8, 8, 4 + idx2) <= idx2, 0, 85),
       If(If(4 + idx2 <= 8, 8, 4 + idx2) <= 1 + idx2, 0, 85),
       If(If(4 + idx2 <= 8, 8, 4 + idx2) <= 2 + idx2, 0, 85),
       If(If(4 + idx2 <= 8, 8, 4 + idx2) <= 3 + idx2, 0, 85))

In [63]:
s = Solver()
s.add(idx2 == 0)
s.check()
print(0x55555555)
print(s.model().eval(val))

1431655765
1431655765


In [64]:
s = Solver()
s.add(idx2 == 0)
s.check()
print(0x55555522)
print(s.model().eval(mload(1)))

1431655714
1431655714


### Bit vector key value store (EVM storage)

In [72]:
storage = K(BitVecSort(32), BitVecVal(0, 32))

def sstore(idx, val):
    global storage    
    
    if isinstance(val, str):
        val = int.from_bytes(bytes.fromhex(val), byteorder='big')
        
    storage = Store(storage, idx, val)        

def sload(idx):
    return storage[idx]

In [73]:
print(simplify(sload(0)))
print()

sstore(0, '11' * 4)
print(0x11111111)
print(simplify(sload(0)))
print()

sstore(1, '22' * 4)
print(0x11111111)
print(simplify(sload(0)))
print()
print(0x22222222)
print(simplify(sload(1)))
print()

0

286331153
286331153

286331153
286331153

572662306
572662306



In [77]:
val = BitVec('val', 32)
sstore(0, val)
simplify(sload(0))

val

In [75]:
idx = BitVec('idx', 32)
val = sload(idx)

Store(Store(Store(K(BitVec(32), 0), 0, 286331153),
            1,
            572662306),
      0,
      x)[idx]

In [76]:
s = Solver()
s.add(idx == 1)
s.check()
print(0x22222222)
print(s.model().eval(sload(idx)))

572662306
572662306
